CS 677 Final Project  
Sajin Shajee  
Sunny Gajjar  
Ahmed Arafaath  
Rohit Subramanian   

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from typing import Dict, Text
from tensorflow import keras
from tqdm.notebook import tqdm
tqdm.pandas()
import tensorflow_recommenders as tfrs
try:
    if not tf.config.list_physical_devices('GPU'):
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")
except:
    if not tf.test.is_gpu_available():
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
try:
    if not tf.config.list_physical_devices('GPU'):
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)
except:
    if not tf.test.is_gpu_available():
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


# Function zero_f  
input: Rows of the dataframe.</br> Output: Converted features with standardized length of 10 characters.
_______  
Description: Converts customer id to string then adds to the right until the length of id is 10 charaters long.

In [2]:
def zero_f(item):
    item=str(item)
    tem=len(item)
    if(len(item)<10):
        item=item.zfill(10)
    return item

# Data Loading and Preprocessing with tensorflow

In [3]:
# Load transaction data 
trans_train = pd.read_csv('h-and-m-personalized-fashion-recommendations/transactions_train.csv',dtype={'customer_id': str,'article_id':str})
trans_train['quantity']=1
trans_train = trans_train[trans_train['t_dat'] >'2020-08-01']
# Load article and customers data
articles= pd.read_csv('h-and-m-personalized-fashion-recommendations/articles.csv',dtype={'article_id': str,'product_code':str})
customers = pd.read_csv('h-and-m-personalized-fashion-recommendations/customers.csv',dtype={'customer_id':str})
master_df = trans_train[['customer_id','article_id','t_dat']].astype(str)
master_df['article_id']=master_df['article_id'].apply(zero_f)
master_df['quantity'] = trans_train['quantity'].astype(float)
masterdf = master_df
len(np.unique(trans_train['customer_id']))

359222

In [4]:
masterdf.head(10)

,customer_id,article_id,t_dat,quantity
29794821,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0853474004,2020-08-02,1.0
29794822,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865594003,2020-08-02,1.0
29794823,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865699001,2020-08-02,1.0
29794824,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0822959001,2020-08-02,1.0
29794825,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0888024005,2020-08-02,1.0
29794826,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0817361007,2020-08-02,1.0
29794827,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,0784053005,2020-08-02,1.0
29794828,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,0873217004,2020-08-02,1.0
29794829,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,0873217004,2020-08-02,1.0
29794830,000d3ae6f3d047f12c5caa2c0125f7f6bb0475c0bdc3e2...,0838340003,2020-08-02,1.0


In [5]:
articles.head(10)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,0108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,0108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,0108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
5,0110065011,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,12,Light Beige,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,0111565001,0111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
7,0111565003,0111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,0111586001,0111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
9,0111593001,0111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."


In [6]:
customers.head(10)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,NaN,NaN,NaN,NaN,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,NaN,NaN,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


In [7]:
# Feature Transformation
interactions = masterdf
interactions['t_dat']=pd.to_datetime(interactions['t_dat'])
train = interactions[interactions['t_dat']<='2020-09-15']
valid = interactions[(interactions['t_dat'] <='2019-09-17')&(interactions['t_dat'] >'2019-09-15')]
test = interactions[interactions['t_dat'] >'2019-09-17']
print(len(np.unique(train['customer_id'])))
#Batch processing and loading with tensorflow
train_ds = tf.data.Dataset.from_tensor_slices(dict(train[['customer_id','article_id']])).shuffle(100_000).batch(256).cache()
valid_ds = tf.data.Dataset.from_tensor_slices(dict(valid[['customer_id','article_id']])).batch(256).cache()
test_ds = tf.data.Dataset.from_tensor_slices(dict(test[['customer_id','article_id']])).batch(256).cache()

329034


In [8]:
#Feature Engineering
items_dict = articles[['article_id']].drop_duplicates()
customer_dict=customers[['customer_id']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
customer_dict={name:np.array(value) for name,value in customer_dict.items()}
customers=tf.data.Dataset.from_tensor_slices(customer_dict)
items = tf.data.Dataset.from_tensor_slices(items_dict)
items = items.map(lambda x: x['article_id'])
customers=customers.map(lambda x: x['customer_id'])

In [9]:
### get unique item and user id's as a lookup table
unique_items = np.unique(np.concatenate(list(items.batch(1_000))))
unique_user_ids = np.unique(np.concatenate(list(customers.batch(1_000))))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)

# Function my_l1_regularizer and my_positive_weights  
Input: Trainable weights  
Output: Updated weights
___
Description: Converts input weights to positive value and regularized value

In [10]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))
def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

Combines user model with item model to generate best recommendations

In [11]:
class CandidateModel(tfrs.Model):

    def __init__(self, user_model, item_model):
        super().__init__()
        
        self.item_model: tf.keras.Model = item_model
        self.user_model: tf.keras.Model = user_model
        
        ### for retrieval model. we take top-k accuracy as metrics
        metrics = tfrs.metrics.FactorizedTopK(candidates=items.batch(256).map(item_model))
        
        # define the task, which is retrieval                                    )    
        task = tfrs.tasks.Retrieval(metrics=metrics)
       
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["customer_id"])

        article_embeddings = self.item_model(features["article_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, article_embeddings)

# Model training  
Orignal model

In [12]:
### Fitting and evaluating

### we choose the dimensionality of the query and candicate representation.
embedding_dimension = 64

## we pass the model, which is the same model we created in the query and candidate tower, into the model
item_model = tf.keras.Sequential([
                                tf.keras.layers.StringLookup(
                                vocabulary=unique_items, mask_token=None),
                                tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension)
                                ])

user_model = tf.keras.Sequential([
                                tf.keras.layers.StringLookup(
                                vocabulary=unique_user_ids, mask_token=None),
                                # We add an additional embedding to account for unknown tokens.
                                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
                                ])

model_1 = CandidateModel(user_model, item_model)
model_1.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
model_1.fit(train_ds,epochs=3,batch_size=128)
model_1.evaluate(test_ds, return_dict=True)

Epoch 1/3
6849/6849 [==============================] - 2834s 414ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0630 - factorized_top_k/top_10_categorical_accuracy: 0.0886 - factorized_top_k/top_50_categorical_accuracy: 0.1632 - factorized_top_k/top_100_categorical_accuracy: 0.2086 - loss: 1297.0611 - regularization_loss: 0.0000e+00 - total_loss: 1297.0611
Epoch 2/3
6849/6849 [==============================] - 2786s 407ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0230 - factorized_top_k/top_5_categorical_accuracy: 0.2445 - factorized_top_k/top_10_categorical_accuracy: 0.3180 - factorized_top_k/top_50_categorical_accuracy: 0.4708 - factorized_top_k/top_100_categorical_accuracy: 0.5381 - loss: 906.3326 - regularization_loss: 0.0000e+00 - total_loss: 906.3326
Epoch 3/3
7788/7788 [==============================] - 2562s 329ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1235 - factorized_top_k/top_5_categor

{'factorized_top_k/top_1_categorical_accuracy': 0.12352025508880615,
 'factorized_top_k/top_5_categorical_accuracy': 0.4258468747138977,
 'factorized_top_k/top_10_categorical_accuracy': 0.5183212757110596,
 'factorized_top_k/top_50_categorical_accuracy': 0.6664695143699646,
 'factorized_top_k/top_100_categorical_accuracy': 0.7158278822898865,
 'loss': 132.26890563964844,
 'regularization_loss': 0,
 'total_loss': 132.26890563964844}

# Function decoder and run_f 
Input: feature from submission file  
Output: decoded and vectorized variable  
_____________

In [13]:
def decoder(e):
    return e.decode('UTF-8')
def run_f(item):
    _, titles = index(tf.constant([item]),k=12)
    t = np.array(titles[0])
    vfunc = np.vectorize(decoder)
    l = vfunc(t)
    l = " ".join(l)
    return l

This will enable prediction for each customers.

In [14]:
index = tfrs.layers.factorized_top_k.BruteForce(model_1.user_model)
index.index_from_dataset(items.batch(100).map(lambda items: (items,model_1.item_model(items))))

This indicates the process that was used to convert values for each customer into a string of values. These values are added into the csv file. TQDM provides a progress bar that indicates the progress of the transformations indicated with the apply function.

In [15]:
submission_file = pd.read_csv('h-and-m-personalized-fashion-recommendations/sample_submission.csv',dtype={'customer_id': str})
sub_cust = submission_file["customer_id"]
sub_df = pd.DataFrame(columns=['Customer_Id', 'Article_Id'])
submission_file["prediction"] = submission_file['customer_id'].progress_apply(run_f)
submission_file.head()

  0%|          | 0/1371980 [00:00<?, ?it/s]

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0858856005 0762846028 0779781006 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0688537021 0640021011 0687034023 0803757004 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321011 0805000001 0805000007 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0878097002 0706016015 0832473003 0878097001 08...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0791587001 0868195004 0791587015 0791587009 07...


In [16]:
submission_file.to_csv('submission_1.csv',index=False)

# Model Improvement

In [17]:
### Fitting and evaluating

### we choose the dimensionality of the query and candicate representation.
embedding_dimension = 64

## we pass the model, which is the same model we created in the query and candidate tower, into the model
item_model = tf.keras.Sequential([
                                tf.keras.layers.StringLookup(
                                vocabulary=unique_items, mask_token=None),
                                tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense((embedding_dimension),activation="softmax",kernel_regularizer=my_l1_regularizer,kernel_constraint=my_positive_weights)
                                ])

user_model = tf.keras.Sequential([
                                tf.keras.layers.StringLookup(
                                vocabulary=unique_user_ids, mask_token=None),
                                # We add an additional embedding to account for unknown tokens.
                                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense((embedding_dimension),activation="relu",kernel_initializer='he_normal',use_bias=False),
                                tf.keras.layers.BatchNormalization(),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense((embedding_dimension),kernel_regularizer=my_l1_regularizer,kernel_constraint=my_positive_weights)
                                ])

model_2 = CandidateModel(user_model, item_model)
model_2.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
model_2.fit(train_ds,validation_data=valid_ds,epochs=3,batch_size=128)
model_2.evaluate(test_ds, return_dict=True)

Epoch 1/3
6849/6849 [==============================] - 2552s 372ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3936 - factorized_top_k/top_5_categorical_accuracy: 0.3936 - factorized_top_k/top_10_categorical_accuracy: 0.5192 - factorized_top_k/top_50_categorical_accuracy: 0.5193 - factorized_top_k/top_100_categorical_accuracy: 0.5194 - loss: 1419.2926 - regularization_loss: 0.0157 - total_loss: 1419.3083
Epoch 2/3
6849/6849 [==============================] - 2524s 369ms/step - factorized_top_k/top_1_categorical_accuracy: 0.4000 - factorized_top_k/top_5_categorical_accuracy: 0.4004 - factorized_top_k/top_10_categorical_accuracy: 0.5590 - factorized_top_k/top_50_categorical_accuracy: 0.5592 - factorized_top_k/top_100_categorical_accuracy: 0.5592 - loss: 1419.2926 - regularization_loss: 1.0286e-07 - total_loss: 1419.2926
Epoch 3/3
7788/7788 [==============================] - 3158s 405ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0052 - factorized_top_k/top_5_categoric

{'factorized_top_k/top_1_categorical_accuracy': 0.005231996066868305,
 'factorized_top_k/top_5_categorical_accuracy': 0.005231996066868305,
 'factorized_top_k/top_10_categorical_accuracy': 0.12519118189811707,
 'factorized_top_k/top_50_categorical_accuracy': 0.12519118189811707,
 'factorized_top_k/top_100_categorical_accuracy': 0.12519118189811707,
 'loss': 106.45360565185547,
 'regularization_loss': 3.531923908894896e-08,
 'total_loss': 106.45360565185547}

Regularization and constraints are implemented here. Despite the accuracy dropping, the loss has been significantly reduced due to the improvements that were implemented.

In [18]:
# bucket list of unique items
unique_items

array([b'0108775015', b'0108775044', b'0108775051', ..., b'0956217002',
       b'0957375001', b'0959461001'], dtype=object)

In [19]:
#bucket list of unique customers
unique_user_ids 

array([b'00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       b'0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa',
       b'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       ...,
       b'ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264',
       b'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
       b'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
      dtype=object)

In [20]:
index = tfrs.layers.factorized_top_k.BruteForce(model_2.user_model)
index.index_from_dataset(items.batch(100).map(lambda items: (items,model_2.item_model(items))))

Sample prediction

In [21]:
j = '000231cc9af9e58ab4edc66fbd61da921b144ba85bc1c00d0ae2309531e4c210'
_, titles = index(tf.constant([j]),k=12)
print(f"Recommendations for user %s: {titles[0]}" %(j))

Recommendations for user 000231cc9af9e58ab4edc66fbd61da921b144ba85bc1c00d0ae2309531e4c210: [b'0108775015' b'0108775044' b'0108775051' b'0110065001' b'0110065002'
 b'0110065011' b'0111565001' b'0111565003' b'0111586001' b'0111593001'
 b'0111609001' b'0112679048']


In [22]:
submission_file = pd.read_csv('h-and-m-personalized-fashion-recommendations/sample_submission.csv',dtype={'customer_id': str})
sub_cust = submission_file["customer_id"]
sub_df = pd.DataFrame(columns=['Customer_Id', 'Article_Id'])
submission_file["prediction"] = submission_file['customer_id'].progress_apply(run_f)
submission_file.head()

  0%|          | 0/1371980 [00:00<?, ?it/s]

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0108775015 0108775044 0108775051 0110065001 01...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0108775015 0108775044 0108775051 0110065001 01...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0108775015 0108775044 0108775051 0110065001 01...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0108775015 0108775044 0108775051 0110065001 01...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0108775015 0108775044 0108775051 0110065001 01...


In [23]:
submission_file.to_csv('submission_2.csv',index=False)